# Exploratory Analysis of Competitor Reviews

In [26]:
import os
import pandas as pd
from keybert import KeyBERT
from bertopic import BERTopic
import glob

## Load The Dataset

**This can be replaced with a script to automatically pull the reviews**

In [2]:
file_name = "B072JK37LV.csv"

In [3]:
df = pd.read_csv(os.path.join('..', 'data', file_name), error_bad_lines=False)

b'Skipping line 24: expected 6 fields, saw 7\nSkipping line 25: expected 6 fields, saw 7\nSkipping line 36: expected 6 fields, saw 7\nSkipping line 43: expected 6 fields, saw 7\n'


In [4]:
df.head()

,ReviewerName,Stars,ReviewLink,Title,Description,Review
0,Aly Fam,5.0,https://www.amazon.com//gp/customer-reviews/RI...,Works Great with Lenovo Yoga 920 and x2 4K LG ...,Reviewed in the United States on February 20 ...,Works with my Lenovo Yoga 920. I have x2 4K LG...
1,Christian Carvajal,1.0,https://www.amazon.com//gp/customer-reviews/R1...,Doesn’t work on MacBook Air M1,Reviewed in the United States on January 28 2021,MacBook Air M1 here. Didn’t work at all. Not e...
2,Steven R. Sloshberg,5.0,https://www.amazon.com//gp/customer-reviews/R3...,Great way to add second monitor to desktop PC,Reviewed in the United States on January 4 2021,Fantastic add on for desktop PC. Acts as a sec...
3,Amazon Customer,5.0,https://www.amazon.com//gp/customer-reviews/R1...,Awesome technical customer service.,Reviewed in the United States on November 24 ...,I bought this item because their technical ser...
4,BIG O,5.0,https://www.amazon.com//gp/customer-reviews/R3...,Real deal,Reviewed in the United States on November 15 ...,This is the real deal. Plug it and forget it.


In [5]:
one_star = df.loc[df['Stars'] < 2] 
one_star.head()

,ReviewerName,Stars,ReviewLink,Title,Description,Review
1,Christian Carvajal,1.0,https://www.amazon.com//gp/customer-reviews/R1...,Doesn’t work on MacBook Air M1,Reviewed in the United States on January 28 2021,MacBook Air M1 here. Didn’t work at all. Not e...
21,Ryan,1.0,https://www.amazon.com//gp/customer-reviews/R1...,Simply Doesn't Work,Reviewed in the United States on September 26 ...,Macbook Pro 2017 15\ 3.1 Ghz top-end at the ti...
23,C Family,1.0,https://www.amazon.com//gp/customer-reviews/RU...,Was great while it lasted,Reviewed in the United States on March 9 2018,worked for 2 months then stopped working. Was ...
27,NOYGDB,1.0,https://www.amazon.com//gp/customer-reviews/R1...,Razer Pro- Doesn't work.,Reviewed in the United States on February 12 ...,Doesn't work on new Razer Pro FHD1060. Downloa...
33,Stephanie,1.0,https://www.amazon.com//gp/customer-reviews/R3...,Doesn't work with Cinema 4K displays,Reviewed in the United States on September 2 ...,Does not work with my brand new 2017 MacBook P...


In [6]:
four_five_star = df.loc[df['Stars'] >= 4] 
four_five_star.head()

,ReviewerName,Stars,ReviewLink,Title,Description,Review
0,Aly Fam,5.0,https://www.amazon.com//gp/customer-reviews/RI...,Works Great with Lenovo Yoga 920 and x2 4K LG ...,Reviewed in the United States on February 20 ...,Works with my Lenovo Yoga 920. I have x2 4K LG...
2,Steven R. Sloshberg,5.0,https://www.amazon.com//gp/customer-reviews/R3...,Great way to add second monitor to desktop PC,Reviewed in the United States on January 4 2021,Fantastic add on for desktop PC. Acts as a sec...
3,Amazon Customer,5.0,https://www.amazon.com//gp/customer-reviews/R1...,Awesome technical customer service.,Reviewed in the United States on November 24 ...,I bought this item because their technical ser...
4,BIG O,5.0,https://www.amazon.com//gp/customer-reviews/R3...,Real deal,Reviewed in the United States on November 15 ...,This is the real deal. Plug it and forget it.
6,Apple0727,5.0,https://www.amazon.com//gp/customer-reviews/R3...,"3 monitors 3 different screens, this is the p...",Reviewed in the United States on August 26 2020,A little expensive but so happy with product ...


### Applying KeyBERT

In [29]:
doc_list = list(four_five_star['Review'])
doc = "\n".join(doc_list)
# doc

In [20]:
kw_model = KeyBERT('distilbert-base-nli-mean-tokens')
keywords = kw_model.extract_keywords(doc, keyphrase_ngram_range=(2, 8), stop_words='english', 
                                      use_mmr=True, nr_candidates=10)

In [22]:
keywords

[('lenovo yoga 920 x2 4k lg monitors hooked', 0.7263),
 ('slight flickering monitors initially laptop connection loose', 0.5439),
 ('expensive happy product does exactly need connected macbook', 0.6127),
 ('bootcamp nvidia driver folder manually picks right works', 0.5958),
 ('mojave win10 bootcamp works perfectly lag just smooth', 0.5692)]

### Applying BertTopic

In [80]:
from flair.embeddings import TransformerDocumentEmbeddings

amazon_reviews_zh = TransformerDocumentEmbeddings('activebus/BERT_Review')
# topic_model = BERTopic(embedding_model=roberta)

Downloading:   0%|          | 0.00/499 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/534M [00:00<?, ?B/s]

Some weights of the model checkpoint at activebus/BERT_Review were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [81]:
topic_model = BERTopic(embedding_model=amazon_reviews_zh, nr_topics=2, n_gram_range=(1, 3), min_topic_size=5)
topic_model.fit(doc_list)

In [82]:
topics, _ = topic_model.transform(doc_list); topics

array([ 1, -1,  1,  0,  1,  1,  1,  1,  1,  1,  0,  1,  1,  1,  1,  1,  1,
        1,  1,  0,  1,  0,  0, -1, -1,  1,  0,  1, -1,  1])

In [83]:
topic_model.get_topic_info()

,Topic,Count,Name
0,1,20,1_with_is_thunderbolt_monitors
1,0,6,0_no issues_good no issues_far_far plug
2,-1,4,-1_great very pleased_great very_second video_...
